# Numerical Analysis Final Project: Lasso vs Ridge Regression

In [2]:
import Pkg
Pkg.add("Pandas")
Pkg.add("LinearAlgebra")
Pkg.add("Plots")

    Updating registry at `C:\Users\Kingston\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Kingston\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Kingston\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Kingston\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Kingston\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Kingston\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Kingston\.julia\environments\v1.7\Manifest.toml`


In [3]:
using Pkg
using Pandas
using LinearAlgebra
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1423

(process:15848): GLib-GIO-WARNING **: 14:32:05.404: Unexpectedly, UWP app `Evernote.Evernote_10.33.5.0_x86__q4d96b2w5wcc2' (AUMId `Evernote.Evernote_q4d96b2w5wcc2!Evernote') supports 1 extensions but has no verbs
ERROR: LoadError: ArgumentError: not a path: `C:\Users\Kingston\.julia\packages\GR\KPElO\deps\deps.jl`
Stacktrace:
 [1] RelocatableFolders.Path(mod::Module, dir::String, path::String)
   @ RelocatableFolders C:\Users\Kingston\.julia\packages\RelocatableFolders\PYUl8\src\RelocatableFolders.jl:51
 [2] top-level scope
   @ C:\Users\Kingston\.julia\packages\GR\KPElO\src\GR.jl:36
 [3] include
   @ .\Base.jl:418 [inlined]
 [4] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::String)
   @ Base .\loading.jl:1318
 [5] top-level scope
   @ none:1


LoadError: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to C:\Users\Kingston\.julia\compiled\v1.7\Plots\jl_7AF9.tmp.

## Import data

In [5]:
df = read_csv("median_housing_cost_data.tsv", sep="\t");
df_targ = read_csv("housing_cost_targets.tsv", sep="\t");

In [6]:
x = iloc(df)[0]

Unnamed: 0    20639.000000
MedInc            2.388600
HouseAge         16.000000
AveRooms          5.254717
AveBedrms         1.162264
Population     1387.000000
AveOccup          2.616981
Latitude         39.370000
Longitude      -121.240000
Name: 20639, dtype: float64

In [10]:
y = iloc(df_targ)[0]

Unnamed: 0     20639.000
House Price        0.894
Name: 20639, dtype: float64

## Motivating factor

In [4]:
# talk about median household costs and how maximizing posterior probabilities to estimate cost is goal

## Regression tasks derived from maximizing posterior probabilities

In [18]:
# markdown derivations put here

## Solutions to regression tasks

In [19]:
# markdown derivations put here

## Code solutions for regression tasks

In [5]:
function linear_regression_ridge(X,y,lambda)
    s = size(X)[2]
    Im =1* Matrix(I, s, s)
    
    w = inv(transpose(X)*X + lambda*Im) * transpose(X) * y
    
    pred = X * w
    
    err = y - pred
    
    return w, pred, err    
    
end

linear_regression_ridge (generic function with 1 method)

In [10]:
function linear_regression_lasso(X,y,lambda)
    
    moore_pen = inv(transpose(X)*X) * transpose(X)
    w = moore_pen * y
    
    w_lasso = sign.(w) .* (abs.(w) .- lambda)
    
    pred = X * w_lasso
    
    err = y - pred
    
    return w_lasso, pred, err
end

linear_regression_lasso (generic function with 1 method)

In [13]:
#for performing linear regression on some data X with provided coefficients w
function linear_regression_test(X,w)
    return X * w        # prediction
end

linear_regression_test (generic function with 1 method)

## Analysis of algorithms: conditioning, complexities, and flop counts

------

### Time Complexity

#### Ridge Regression

In [14]:
# Ridge Regression Function Decomposed to show computations:
# Parameters- X: (m x n), y: (m x 1), lambda: constant
function ridge_TC(X,y,lambda)
    s  = size(X)[2]                          # O(1)      [CHANGED [1] to [2]... Correct (grab m or n)?]
    Im = 1 * Matrix(I, s, s)                 # O(n^2)    [Constructing (n x n) I Matrix]
    xT = transpose(X)                        # O(1)      [(m x n)] => (n x m)
    xT_X = xT * X                            # O(m^2*n)  [(n x m) * (m x n)] => (n x n)
    l_Im = lambda * Im                       # O(n^2)    [Scalar multiplication of (n x n) matrix]
    xT_lambda = l_Im + xT_X                  # O(n^2)    [Scalar addition of (n x n) matrix]
    inverse = inv(xT_lambda)                 # O(n^3)    [Inverse (Gauss Elimination) with matrix of size (n x n)]
    w = inverse * xT                         # O(n^2*m)  [(n x n) * (n x m)] => (n x m)
    w *= y                                   # O(m*n)    [(n x m) * (m x 1)] => (n x 1)
    pred = X * w                             # O(m*n)    [(m x n) * (n x 1)] => (m x 1)
    err = y - pred                           # O(m)      [(m x 1) - (m x 1)] => (m x 1)
    
    return w, pred, err                      # O(1)
end

linear_regression_ridge (generic function with 1 method)

In [15]:
# Time Complexities Added
# O(n^3) + O(n^2*m) + O(m^2*n) + O(n^2) + O(n^2) + O(n^2) + O(m*n) + O(m*n) + O(m) + O(1) + O(1) + O(1)

# Final Time Complexity: 
# O(n^3) + O(n^2*m) + O(m^2*n)

# Note: If m >> n (Data points >> Features), time complexity can be reduced to O(m^2*n)

In [12]:
# Variable Dimensions
# --------------------------------
# lambda = constant
# (n x n) = xT_X, inverse, Im, l_Im xT_lambda
# (n x m) = xT
# (n x 1) = w
# (m x n) = X
# (m x 1) = y, pred, err

#### Lasso Regression

In [ ]:
# Lasso Regression Function Decomposed to show computations:
# Parameters- X: (m x n), y: (m x 1), lambda: constant
function lasso_TC(X,y,lambda)  
    xT = transpose(X)              # O(1)        [(m x n)] => (n x m)
    xT_X = xT * X                  # O(m^2*n)    [(n x m) * (m x n)] => (n x n)
    xInv = inv(xT_X)               # O(n^3)      [Inverse (Gauss Elimination) with matrix of size n]
    moore_pen = xInv * xT          # O(n^2*m)    [(n x n) * (n x m)] => (n x m)
    w = moore_pen * y              # O(m*n)      [(n x m) * (m x 1)] => (n x 1)
    w_lam = (abs.(w) .- lambda)    # O(n)        [Scalar subtraction of (n x 1) matrix]
    w_lasso = sign.(w) .* w_lam    # O(n)        [Scalar multiplication of (n x 1) matrix]
    pred = X * w_lasso             # O(m*n)      [(m x n) * (n x 1)] => (m x 1)
    err = y - pred                 # O(m)        [(m x 1) - (m x 1)] => (m x 1)
    
    return w_lasso, pred, err      # O(1)
end

In [ ]:
# Variable Dimensions
# -------------------------
# lambda = constant
# (n x n) = xT_X
# (n x m) = xT, moore_pen
# (n x 1) = w, w_lam, w_lasso
# (m x n) = X
# (m x 1) = y, pred, err

In [ ]:
# Time Complexity References
# ----------------------------------------
# Size(X)       : O(1)
# References    : [https://stackoverflow.com/questions/21614298/what-is-the-runtime-of-array-length, https://blog.finxter.com/python-list-length-whats-the-runtime-complexity-of-len/, ]

# Transpose(X)  : O(1)    
# References    : [https://www.mathworks.com/matlabcentral/answers/495668-what-s-the-transpose-complexity-big-o-in-matlab, https://stackoverflow.com/questions/61157101/in-julia-transpose-operator]

# Inverse(X)    : Worst Case-O(n^3) (Gauss Elimination), Best Case O(n^2.373)
# References    : [https://stackoverflow.com/questions/54890422/inv-versus-on-julia]

# Matrix *      : (m x n) * (n * p) => O(n*m*p), O(n^3)-O(n^2.72...)
# References    : [https://en.wikipedia.org/wiki/Computational_complexity_of_mathematical_operations#Matrix_algebra]

# Matrix -      : O(m*n)
# References    : [https://www.geeksforgeeks.org/different-operation-matrices/]

# Matrix(I,s,s) : O(n^2)
# References    : []

### Space Complexity

#### Ridge Regression

In [18]:
# Actual Ridge Algorithm
function ridge_SC(X,y,lambda)
    n = size(X)[2]                                          # O(1)    1 variable
    Im = 1 * Matrix(I, n, n)                                # O(n^2)  (n x n) matrix
    w = inv(transpose(X)*X + lambda*Im) * transpose(X) * y  # O(n)    (n x 1) matrix
    pred = X * w                                            # O(m)    (m x 1) matrix
    err = y - pred                                          # O(m)    (m x 1) matrix
    
    return w, pred, err                                     # O(1)    Return on Stack
end

ridge_SC (generic function with 1 method)

In [17]:
# Total Space Complexity
# O(n^2) + O(n) + O(m) + O(m) + O(1) + O(1)

# O(n^2) + O(m)

#### Lasso Regression

In [ ]:
# Actual Ridge Algorithm
function lasso_SC(X,y,lambda)
    moore_pen = inv(transpose(X)*X) * transpose(X)    # O(m*n) (n x m) matrix
    w = moore_pen * y                                 # O(n)   (n x 1) matrix
    w_lasso = sign.(w) .* (abs.(w) .- lambda)         # O(n)   (n x 1) matrix
    pred = X * w_lasso                                # O(m)   (m x 1) matrix
    err = y - pred                                    # O(m)   (m x 1) matrix
    
    return w_lasso, pred, err                         # O(1)   Return on Stack
end

In [ ]:
# Total Space Complexity
# O(m*n) + O(n) + O(n) + O(m) + O(m) + O(1)

# O(m*n)

In [ ]:
# Space Complexity References
# -------------------------------
# [Space Complexity Calculates Temp Vars?] https://www.studytonight.com/data-structures/space-complexity-of-algorithms

### Flop Counts

#### Ridge Regression

In [ ]:
# Ridge Regression Function Decomposed:
# Parameters- X: (m x n), y: (m x 1), lambda: constant
function ridge_FC(X,y,lambda)
    n  = size(X)[2]                          # 0
    Im = 1 * Matrix(I, n, n)                 # n^2          [Constructing (n x n) I Matrix]
    xT = transpose(X)                        # 0            [(m x n)] => (n x m)
    xT_X = xT * X                            # m^2*n        [(n x m) * (m x n)] => (n x n)
    l_Im = lambda * Im                       # n^2          [Scalar multiplication of (n x n) matrix]
    xT_lambda = l_Im + xT_X                  # n^2          [Scalar addition of (n x n) matrix]
    inverse = inv(xT_lambda)                 # n^3 (approx) [Gauss Elimination with matrix of size (n x n)]
    w = inverse * xT                         # n^2*m        [(n x n) * (n x m)] => (n x m)
    w *= y                                   # m*n          [(n x m) * (m x 1)] => (n x 1)
    pred = X * w                             # m*n          [(m x n) * (n x 1)] => (m x 1)
    err = y - pred                           # m            [(m x 1) - (m x 1)] => (m x 1)
    
    return w, pred, err                      # 0
end

In [ ]:
# Flop Count
# n^2 + (m^2*n) + n^2 + n^2 + n^3 + n^2*m + m*n + m*n + m
# Total Flop Count = n^3 + (3+m)n^2 + n*m^2 + 2m*n + m

# Note: In the worst case, Gaussian Elimination for n x n will take ((5/6)n^3+(3/2)n^2-(7/6)n) floating point operations.
# Worst Case Flops = (5/6)n^3 + ((9/2)+m)n^2 + n*m^2 + ((5/6)m)n + m

#### Lasso Regression

In [ ]:
# Ridge Regression Function Decomposed:
# Parameters- X: (m x n), y: (m x 1), lambda: constant
function lasso_TC(X,y,lambda)  
    xT = transpose(X)              # 0
    xT_X = xT * X                  # m^2*n        [(n x m) * (m x n)] => (n x n)
    xInv = inv(xT_X)               # n^3 (approx) [Gauss Elimination with matrix of size (n x n)]
    moore_pen = xInv * xT          # n^2*m        [(n x n) * (n x m)] => (n x m)
    w = moore_pen * y              # m*n          [(n x m) * (m x 1)] => (n x 1)
    w_lam = (abs.(w) .- lambda)    # n            [Scalar subtraction of (n x 1) matrix]
    w_lasso = sign.(w) .* w_lam    # n            [Scalar multiplication of (n x 1) matrix]
    pred = X * w_lasso             # m*n          [(m x n) * (n x 1)] => (m x 1)
    err = y - pred                 # m            [(m x 1) - (m x 1)] => (m x 1)
    
    return w_lasso, pred, err      # 0
end

In [20]:
# Flop Count
# Flops = n^3 + n*m^2 + m*n^2 + 2m*n + 2n + m

# Note: In the worst case, Gaussian Elimination for n x n will take ((5/6)n^3+(3/2)n^2-(7/6)n) floating point operations.
# Worst Case Flops = (5/6)n^3 + n*m^2 + (3/2 + m)*n^2 + 2m*n + (5/6)n + m

In [ ]:
# Flop Count References:
# Flops For nxn Gaussian Elimination: http://web.mit.edu/18.06/www/Fall15/Matrices.pdf

## Perform regression tasks on data

In [11]:
#extract data from dataframe into julia arrays and then call code solutions of regression

## Evaluation of algorithm performance

In [13]:
# r^2 and qq plot

## Conclusion

In [ ]:
# Our results and conclusions of data